In [1]:
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import keyring
import time

## Setup Spotipy credentials and query wrapper

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id=keyring.get_password('spotify', 'cid'),
                                                      client_secret=keyring.get_password('spotify', 'secret') )
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Get sample artists data

In [3]:
artist_id = '0du5cEVh5yTK9QJze8zA0C'

In [4]:
# View sp.artist output
sp.artist(artist_id)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0du5cEVh5yTK9QJze8zA0C'},
 'followers': {'href': None, 'total': 28322108},
 'genres': ['dance pop', 'pop'],
 'href': 'https://api.spotify.com/v1/artists/0du5cEVh5yTK9QJze8zA0C',
 'id': '0du5cEVh5yTK9QJze8zA0C',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/c1ee384b2fe0ea8e9df0b194a17d6b41b2b7993a',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/aba91de7087e3b657cf11e98c45026ac6b7df544',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/b958dfcbcfed9af560764c04c4afc19381744892',
   'width': 160}],
 'name': 'Bruno Mars',
 'popularity': 88,
 'type': 'artist',
 'uri': 'spotify:artist:0du5cEVh5yTK9QJze8zA0C'}

## Get sample track data

In [5]:
track_id = "0ofbQMrRDsUaVKq2mGLEAb"

In [6]:
# View sp.track output
sp.track(track_id)

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
    'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
    'id': '4nDoRrQiYLoBzwC5BhVJzF',
    'name': 'Camila Cabello',
    'type': 'artist',
    'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/5chBPOVY2I0bG5V3igb5QL'},
  'href': 'https://api.spotify.com/v1/albums/5chBPOVY2I0bG5V3igb5QL',
  'id': '5chBPOVY2I0bG5V3igb5QL',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273d93cf4d07ba50d7b32ca7c44',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02d93cf4d07ba50d7b32ca7c44',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851d93cf4d07ba50d7b32ca7c44',
    'width': 64}],
  'name': 'Havana',
  'release_date': '2017-08-03',
  'release_date

In [7]:
# View sp.audio_features output
sp.audio_features(track_id)

[{'danceability': 0.768,
  'energy': 0.517,
  'key': 7,
  'loudness': -4.323,
  'mode': 0,
  'speechiness': 0.0312,
  'acousticness': 0.186,
  'instrumentalness': 3.8e-05,
  'liveness': 0.104,
  'valence': 0.418,
  'tempo': 104.992,
  'type': 'audio_features',
  'id': '0ofbQMrRDsUaVKq2mGLEAb',
  'uri': 'spotify:track:0ofbQMrRDsUaVKq2mGLEAb',
  'track_href': 'https://api.spotify.com/v1/tracks/0ofbQMrRDsUaVKq2mGLEAb',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ofbQMrRDsUaVKq2mGLEAb',
  'duration_ms': 216897,
  'time_signature': 4}]

## Read consolidated spotify daily charts

In [8]:
df = pd.read_csv('data/spotify_daily_charts.csv')
df.head()

,date,position,track_id,track_name,artist,streams
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656


## Get data of unique tracks in charts 

In [9]:
def get_track_data(t_id):                    
    track_data = sp.track(t_id)
    track_features = sp.audio_features(t_id)
    
    #get only main(first) artist
    td_list = [t_id,\
               track_data['name'],\
               track_data['artists'][0]['id'],\
               track_data['artists'][0]['name'],\
               track_data['album']['uri'].split(":")[2],\
               track_data['duration_ms'],\
               track_data['album']['release_date'],\
               track_data['popularity']]
    data = pd.DataFrame([td_list], columns = ['track_id','track_name','artist_id','artist_name','album_id','duration','release_date','popularity'])

    relevant_cols = ['danceability', 'energy', 'key', 'loudness', 'mode',\
                     'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']  
    
    tf_data = pd.DataFrame(track_features)
    tf_data = tf_data[relevant_cols]
    
    data = pd.concat([data, tf_data], axis=1)
    return data


In [10]:
get_track_data(track_id)

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0ofbQMrRDsUaVKq2mGLEAb,Havana,4nDoRrQiYLoBzwC5BhVJzF,Camila Cabello,5chBPOVY2I0bG5V3igb5QL,216896,2017-08-03,5,0.768,0.517,7,-4.323,0,0.0312,0.186,0.000038,0.104,0.418,104.992


In [11]:
track_df = df[['track_id','track_name']].drop_duplicates()
track_df

,track_id,track_name
0,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor
1,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go
2,7BKLCZ1jbUBVqRi2FVlTVw,Closer
3,2rizacJSyD9S1IQUxUxnsK,All We Know
4,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know
...,...,...
294138,5X0rdNMk9xtQ5dNVMfZA6x,Sana'y Di Nalang
294373,4Yxc55NX3tAXC2mHRAhtcW,"All I Want - From ""High School Musical: The Mu..."
294420,3jjBPF4NDMCU51psU8JPpR,"34+35 Remix (feat. Doja Cat, Megan Thee Stalli..."
294475,6T6D9CIrHkALcHPafDFA6L,Vibez


In [12]:
len(pd.unique(track_df['track_id'].values)),len(pd.unique(track_df['track_name'].values))

(3284, 2570)

> Q: Why is it that we have fewer unique track names than unique track ids? Is this expected or does it indicate a data processing error?

In [13]:
downloaded_track_ids=[]

In [15]:
track_list = track_df['track_id'].values
df_list=[]

for i,track_id in enumerate(track_list):    
    if track_id in downloaded_track_ids:
        continue
    else:
        print('[%d/%d] Fetching track data for %s... ' % 
              (i+1,len(track_list),track_df[track_df['track_id']==track_id]['track_name'].values[0]), end = " ") 
        track_data = get_track_data(track_id) 

        downloaded_track_ids.append(track_id)
        df_list.append(track_data)

        print('done!')

        # sleep for 100 secs per 100 requests to avoid being blocked
        if (i % 100 == 0)&(i > 0):
            time.sleep(10)    

[1/3288] Fetching track data for Versace on the Floor...  done!
[2/3288] Fetching track data for Say You Won't Let Go...  done!
[3/3288] Fetching track data for Closer...  done!
[4/3288] Fetching track data for All We Know...  done!
[5/3288] Fetching track data for Don't Wanna Know...  done!
[6/3288] Fetching track data for 24K Magic...  done!
[7/3288] Fetching track data for Starving...  done!
[8/3288] Fetching track data for How Far I'll Go - From "Moana"...  done!
[9/3288] Fetching track data for Let Me Love You...  done!
[10/3288] Fetching track data for Starboy...  done!
[11/3288] Fetching track data for Jumpshot...  done!
[12/3288] Fetching track data for Cold Water (feat. Justin Bieber & MØ)...  done!
[13/3288] Fetching track data for Perfect Strangers...  done!
[14/3288] Fetching track data for Bad Things (with Camila Cabello)...  done!
[15/3288] Fetching track data for I Don’t Wanna Live Forever (Fifty Shades Darker) - From "Fifty Shades Darker (Original Motion Picture Soundtr

[131/3288] Fetching track data for Meron Nang Iba...  done!
[132/3288] Fetching track data for Breakeven...  done!
[133/3288] Fetching track data for The Man Who Can't Be Moved...  done!
[134/3288] Fetching track data for Ignition - Remix...  done!
[135/3288] Fetching track data for Terrified...  done!
[136/3288] Fetching track data for Too Good to Say Goodbye...  done!
[137/3288] Fetching track data for Water Under the Bridge...  done!
[138/3288] Fetching track data for Make Me (Cry)...  done!
[139/3288] Fetching track data for Bad and Boujee (feat. Lil Uzi Vert)...  done!
[140/3288] Fetching track data for Sexual...  done!
[141/3288] Fetching track data for More Than Words...  done!
[142/3288] Fetching track data for No Problem (feat. Lil Wayne & 2 Chainz)...  done!
[143/3288] Fetching track data for Some Kind Of Drug (feat. Marc E. Bassy)...  done!
[144/3288] Fetching track data for When You Say Nothing At All...  done!
[145/3288] Fetching track data for Solo Dance...  done!
[146/32

[263/3288] Fetching track data for ILYSB...  done!
[264/3288] Fetching track data for Beautiful...  done!
[265/3288] Fetching track data for Alone...  done!
[266/3288] Fetching track data for Hate That I Love You...  done!
[267/3288] Fetching track data for Magic...  done!
[268/3288] Fetching track data for Run Up (feat. PARTYNEXTDOOR & Nicki Minaj)...  done!
[269/3288] Fetching track data for Scared to Be Lonely...  done!
[270/3288] Fetching track data for So Good (feat. Ty Dolla $ign)...  done!
[271/3288] Fetching track data for I'll Make Love To You...  done!
[272/3288] Fetching track data for On Bended Knee...  done!
[273/3288] Fetching track data for I Miss You...  done!
[274/3288] Fetching track data for Pag-Ibig - From "Dangwa"...  done!
[275/3288] Fetching track data for Despacito (Featuring Daddy Yankee)...  done!
[276/3288] Fetching track data for The Way You Look at Me...  done!
[277/3288] Fetching track data for If I Ain't Got You...  done!
[278/3288] Fetching track data fo

[393/3288] Fetching track data for No Promises (feat. Demi Lovato)...  done!
[394/3288] Fetching track data for Heatstroke (feat. Young Thug, Pharrell Williams & Ariana Grande)...  done!
[395/3288] Fetching track data for HUMBLE....  done!
[396/3288] Fetching track data for Pretty Girl - Cheat Codes X CADE Remix...  done!
[397/3288] Fetching track data for Fix You...  done!
[398/3288] Fetching track data for Hymn for the Weekend...  done!
[399/3288] Fetching track data for TG4M...  done!
[400/3288] Fetching track data for A Head Full of Dreams...  done!
[401/3288] Fetching track data for A Sky Full of Stars...  done!
[402/3288] Fetching track data for Adventure of a Lifetime...  done!
[403/3288] Fetching track data for Up&Up...  done!
[404/3288] Fetching track data for Paradise...  done!
[405/3288] Fetching track data for Viva La Vida...  done!
[406/3288] Fetching track data for Magic...  done!
[407/3288] Fetching track data for Every Teardrop Is a Waterfall...  done!
[408/3288] Fetchi

[523/3288] Fetching track data for At My Best (feat. Hailee Steinfeld)...  done!
[524/3288] Fetching track data for Stay...  done!
[525/3288] Fetching track data for Swish Swish...  done!
[526/3288] Fetching track data for Strip That Down...  done!
[527/3288] Fetching track data for The Way I Are (Dance with Somebody) (feat. Lil Wayne) - Spotify Version...  done!
[528/3288] Fetching track data for Tired...  done!
[529/3288] Fetching track data for I Like Me Better...  done!
[530/3288] Fetching track data for Hands...  done!
[531/3288] Fetching track data for Redbone...  done!
[532/3288] Fetching track data for There for You...  done!
[533/3288] Fetching track data for Strangers...  done!
[534/3288] Fetching track data for Your Song...  done!
[535/3288] Fetching track data for Raincoat (feat. Shy Martin)...  done!
[536/3288] Fetching track data for Stay With You...  done!
[537/3288] Fetching track data for Remember I Told You...  done!
[538/3288] Fetching track data for Down...  done!
[

...

[2699/3288] Fetching track data for THE SCOTTS...  done!
[2700/3288] Fetching track data for If You’re Too Shy (Let Me Know)...  done!
[2701/3288] Fetching track data for If You’re Too Shy (Let Me Know) - Edit...  done!
[2702/3288] Fetching track data for Righteous...  done!
[2703/3288] Fetching track data for Passenger Seat (Acoustic)...  done!
[2704/3288] Fetching track data for Nandito Na...  done!
[2705/3288] Fetching track data for I Choose - From The Netflix Original Film The Willoughbys...  done!
[2706/3288] Fetching track data for Savage Remix (feat. Beyoncé)...  done!
[2707/3288] Fetching track data for Moonlight...  done!
[2708/3288] Fetching track data for Play Date...  done!
[2709/3288] Fetching track data for Passenger Seat...  done!
[2710/3288] Fetching track data for Beautiful Scars...  done!
[2711/3288] Fetching track data for Toosie Slide...  done!
[2712/3288] Fetching track data for Say So (feat. Nicki Minaj)...  done!
[2713/3288] Fetching track data for Pain 1993 (wi

[2831/3288] Fetching track data for How You Like That...  done!
[2832/3288] Fetching track data for God’s Menu...  done!
[2833/3288] Fetching track data for Maria...  done!
[2834/3288] Fetching track data for if this is the last time...  done!
[2835/3288] Fetching track data for Nakikinig Ka Ba Sa Akin...  done!
[2836/3288] Fetching track data for Just Friends...  done!
[2837/3288] Fetching track data for Di Biro...  done!
[2838/3288] Fetching track data for no song without you...  done!
[2839/3288] Fetching track data for la la la that’s how it goes...  done!
[2840/3288] Fetching track data for free love...  done!
[2841/3288] Fetching track data for by my side...  done!
[2842/3288] Fetching track data for dear P...  done!
[2843/3288] Fetching track data for one way to tokyo...  done!
[2844/3288] Fetching track data for loving you is so easy...  done!
[2845/3288] Fetching track data for Go Crazy...  done!
[2846/3288] Fetching track data for can’t bear to be without you...  done!
[2847/

[2962/3288] Fetching track data for MORE & MORE - English Ver....  done!
[2963/3288] Fetching track data for Dynamite - Acoustic Remix...  done!
[2964/3288] Fetching track data for Dynamite - EDM Remix...  done!
[2965/3288] Fetching track data for 24H...  done!
[2966/3288] Fetching track data for you broke me first...  done!
[2967/3288] Fetching track data for ILYSB - STRIPPED...  done!
[2968/3288] Fetching track data for Mood (feat. Iann Dior)...  done!
[2969/3288] Fetching track data for Our First Song...  done!
[2970/3288] Fetching track data for Ice Cream (with Selena Gomez)...  done!
[2971/3288] Fetching track data for THE BADDEST...  done!
[2972/3288] Fetching track data for Over Now (with The Weeknd)...  done!
[2973/3288] Fetching track data for Dynamite - Tropical Remix...  done!
[2974/3288] Fetching track data for Daisies...  done!
[2975/3288] Fetching track data for Never Really Over...  done!
[2976/3288] Fetching track data for Dynamite - Poolside Remix...  done!
[2977/3288]

[3093/3288] Fetching track data for Can't Take My Eyes Off You...  done!
[3094/3288] Fetching track data for Without You...  done!
[3095/3288] Fetching track data for I CAN'T STOP ME...  done!
[3096/3288] Fetching track data for HELL IN HEAVEN...  done!
[3097/3288] Fetching track data for UP NO MORE...  done!
[3098/3288] Fetching track data for DO WHAT WE LIKE...  done!
[3099/3288] Fetching track data for BRING IT BACK...  done!
[3100/3288] Fetching track data for BELIEVER...  done!
[3101/3288] Fetching track data for QUEEN...  done!
[3102/3288] Fetching track data for GO HARD...  done!
[3103/3288] Fetching track data for HANDLE IT...  done!
[3104/3288] Fetching track data for SHOT CLOCK...  done!
[3105/3288] Fetching track data for Blue Hour...  done!
[3106/3288] Fetching track data for DEPEND ON YOU...  done!
[3107/3288] Fetching track data for SAY SOMETHING...  done!
[3108/3288] Fetching track data for BEHIND THE MASK...  done!
[3109/3288] Fetching track data for Ghosting...  done!


[3228/3288] Fetching track data for Afterglow...  done!
[3229/3288] Fetching track data for Copines...  done!
[3230/3288] Fetching track data for Da Best Ang Pasko Ng Pilipino...  done!
[3231/3288] Fetching track data for Wonderful Christmastime - Edited Version / Remastered 2011...  done!
[3232/3288] Fetching track data for The Christmas Song (Shawn Mendes & Camila Cabello)...  done!
[3233/3288] Fetching track data for My Only Wish (This Year)...  done!
[3234/3288] Fetching track data for Driving Home for Christmas - 2019 Remaster...  done!
[3235/3288] Fetching track data for Thank You, Ang Babait Ninyo...  done!
[3236/3288] Fetching track data for Sa Araw Ng Pasko...  done!
[3237/3288] Fetching track data for Naughty List (with Dixie D’Amelio)...  done!
[3238/3288] Fetching track data for A Holly Jolly Christmas - Single Version...  done!
[3239/3288] Fetching track data for Happy Xmas (War Is Over) - Remastered 2010...  done!
[3240/3288] Fetching track data for Oh Santa! (feat. Arian

In [16]:
tracks_data_df = pd.concat(df_list)
tracks_data_df.head()

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,4PgleR09JVnm3zY1fW3XBA,261240,2016-11-17,76,0.578,0.574,2,-6.209,1,0.0454,0.196,0.00000,0.0830,0.301,174.152
0,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,4IWBUUAFIplrNtaOHcJPRM,James Arthur,7oiJYvEJHsmYtrgviAVIBD,211466,2016-10-28,88,0.358,0.557,10,-7.398,1,0.0590,0.695,0.00000,0.0902,0.494,85.043
0,7BKLCZ1jbUBVqRi2FVlTVw,Closer,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,0rSLgV8p5FzfnqlEk4GzxE,244960,2016-07-29,87,0.748,0.524,8,-5.599,1,0.0338,0.414,0.00000,0.1110,0.661,95.010
0,2rizacJSyD9S1IQUxUxnsK,All We Know,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,0xmaV6EtJ4M3ebZUPRnhyb,194080,2016-09-29,71,0.662,0.586,0,-8.821,1,0.0307,0.097,0.00272,0.1150,0.296,90.000
0,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,04gDigrS5kc9YWfZHwBETP,Maroon 5,0fvTn3WXF39kQs9i3bnNpP,214480,2016-10-11,1,0.783,0.623,7,-6.126,1,0.0800,0.338,0.00000,0.0975,0.447,100.048


In [17]:
tracks_data_df.to_csv('data/spotify_daily_charts_tracks.csv', index=False, encoding='utf-8')

In [18]:
tracks_data_df.describe()

,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000,3284.000000
mean,215719.045067,51.722594,0.634441,0.602115,5.131547,-6.767371,0.675700,0.082523,0.272977,0.009924,0.169273,0.458919,118.576819
std,45458.137458,30.225734,0.141448,0.187231,3.605305,2.735238,0.468184,0.081576,0.273128,0.070233,0.130534,0.215052,27.654877
min,37640.000000,0.000000,0.174000,0.025000,0.000000,-24.250000,0.000000,0.023200,0.000002,0.000000,0.021500,0.034900,52.572000
25%,189533.000000,31.000000,0.544750,0.475000,1.000000,-8.138500,0.000000,0.036000,0.045950,0.000000,0.092800,0.291000,97.983250
50%,209862.000000,63.000000,0.651000,0.617000,5.000000,-6.346000,1.000000,0.052000,0.170000,0.000000,0.118000,0.449500,115.943000
75%,237493.000000,74.000000,0.735000,0.744000,8.000000,-4.859000,1.000000,0.089625,0.450000,0.000020,0.200000,0.612000,136.041000
max,536217.000000,100.000000,0.980000,0.978000,11.000000,0.175000,1.000000,0.884000,0.981000,0.908000,0.955000,0.978000,215.090000


## Get data of unique artists in charts 

In [19]:
#Get unique artists id
artist_df = tracks_data_df[['artist_id','artist_name']].drop_duplicates()
artist_df

,artist_id,artist_name
0,0du5cEVh5yTK9QJze8zA0C,Bruno Mars
0,4IWBUUAFIplrNtaOHcJPRM,James Arthur
0,69GGBxA162lTqCwzJG5jLp,The Chainsmokers
0,04gDigrS5kc9YWfZHwBETP,Maroon 5
0,5p7f24Rk5HkUZsaS3BLG5F,Hailee Steinfeld
...,...,...
0,6XpaIBNiVzIetEPCWDvAFP,Whitney Houston
0,4QQgXkCYTt3BlENzhyNETg,"Earth, Wind & Fire"
0,0bBz5bRBkExaej2HxtVfCw,347aidan
0,05YVYeV4HxYp5rrWalvuE1,"Grover Washington, Jr."


In [20]:
len(artist_df)

766

> Q: What does the ratio of unique artists to unique tracks tell you about the nature of the Spotify top-streamed market?

In [21]:
def get_artist_data(a_id):
       
    artist_data = sp.artist(a_id)

    ad_list = [a_id,\
               artist_data['name'],\
               artist_data['followers']['total'],\
               artist_data['genres'],\
               artist_data['popularity']]
    data = pd.DataFrame([ad_list], columns = ['artist_id','artist_name','total_followers','genres','popularity'])

    return data


In [22]:
get_artist_data(artist_id)

,artist_id,artist_name,total_followers,genres,popularity
0,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,28322108,"[dance pop, pop]",88


In [23]:
artist_list = artist_df['artist_id'].values
df_list=[]

for i,artist_id in enumerate(artist_list):
    print('[%d/%d] Fetching artist data for %s... ' % 
          (i+1,len(artist_list),artist_df[artist_df['artist_id']==artist_id]['artist_name'].values[0]), end = " ") 
    artist_data = get_artist_data(artist_id) 
    df_list.append(artist_data)
    print('done!')
    
    #sleep for 100 secs per 100 requests to avoid being blocked
    if (i % 100 == 0)& (i > 0):
        time.sleep(10)   

[1/766] Fetching artist data for Bruno Mars...  done!
[2/766] Fetching artist data for James Arthur...  done!
[3/766] Fetching artist data for The Chainsmokers...  done!
[4/766] Fetching artist data for Maroon 5...  done!
[5/766] Fetching artist data for Hailee Steinfeld...  done!
[6/766] Fetching artist data for Alessia Cara...  done!
[7/766] Fetching artist data for DJ Snake...  done!
[8/766] Fetching artist data for The Weeknd...  done!
[9/766] Fetching artist data for Dawin...  done!
[10/766] Fetching artist data for Major Lazer...  done!
[11/766] Fetching artist data for Jonas Blue...  done!
[12/766] Fetching artist data for Machine Gun Kelly...  done!
[13/766] Fetching artist data for ZAYN...  done!
[14/766] Fetching artist data for The Vamps...  done!
[15/766] Fetching artist data for Martin Garrix...  done!
[16/766] Fetching artist data for Ariana Grande...  done!
[17/766] Fetching artist data for Starley...  done!
[18/766] Fetching artist data for Andy Grammer...  done!
[19/76

[149/766] Fetching artist data for Charice...  done!
[150/766] Fetching artist data for Hoobastank...  done!
[151/766] Fetching artist data for CHANYEOL...  done!
[152/766] Fetching artist data for James Blunt...  done!
[153/766] Fetching artist data for The Fray...  done!
[154/766] Fetching artist data for JP Cooper...  done!
[155/766] Fetching artist data for Beyoncé...  done!
[156/766] Fetching artist data for Daniel Powter...  done!
[157/766] Fetching artist data for Daniel Bedingfield...  done!
[158/766] Fetching artist data for Joshua Garcia...  done!
[159/766] Fetching artist data for KZ Tandingan...  done!
[160/766] Fetching artist data for The Company...  done!
[161/766] Fetching artist data for Callalily...  done!
[162/766] Fetching artist data for Gloc 9...  done!
[163/766] Fetching artist data for Anna Kendrick...  done!
[164/766] Fetching artist data for Lauv...  done!
[165/766] Fetching artist data for Lady Gaga...  done!
[166/766] Fetching artist data for James Bay...  d

[297/766] Fetching artist data for Janine Teñoso...  done!
[298/766] Fetching artist data for Rivermaya...  done!
[299/766] Fetching artist data for Echosmith...  done!
[300/766] Fetching artist data for Cardi B...  done!
[301/766] Fetching artist data for Chris Brown...  done!
[302/766] Fetching artist data for Daniel Caesar...  done!
[303/766] Fetching artist data for CNCO...  done!
[304/766] Fetching artist data for Astrid S...  done!
[305/766] Fetching artist data for Matoma...  done!
[306/766] Fetching artist data for XXXTENTACION...  done!
[307/766] Fetching artist data for Portugal. The Man...  done!
[308/766] Fetching artist data for Pinkfong...  done!
[309/766] Fetching artist data for R3HAB...  done!
[310/766] Fetching artist data for Craig David...  done!
[311/766] Fetching artist data for WALK THE MOON...  done!
[312/766] Fetching artist data for Fall Out Boy...  done!
[313/766] Fetching artist data for Tiësto...  done!
[314/766] Fetching artist data for NF...  done!
[315/7

[443/766] Fetching artist data for Pharrell Williams...  done!
[444/766] Fetching artist data for The 1975...  done!
[445/766] Fetching artist data for Juice WRLD...  done!
[446/766] Fetching artist data for Kanye West...  done!
[447/766] Fetching artist data for The Itchyworms...  done!
[448/766] Fetching artist data for Alec Benjamin...  done!
[449/766] Fetching artist data for MNEK...  done!
[450/766] Fetching artist data for Andrea Babierra...  done!
[451/766] Fetching artist data for 88rising...  done!
[452/766] Fetching artist data for Tyga...  done!
[453/766] Fetching artist data for Daddy Yankee...  done!
[454/766] Fetching artist data for Panic! At The Disco...  done!
[455/766] Fetching artist data for The Carters...  done!
[456/766] Fetching artist data for Ella Mai...  done!
[457/766] Fetching artist data for Jeremy Zucker...  done!
[458/766] Fetching artist data for Boyce Avenue...  done!
[459/766] Fetching artist data for Joseph Vincent...  done!
[460/766] Fetching artist 

[589/766] Fetching artist data for LeeHi...  done!
[590/766] Fetching artist data for Will Smith...  done!
[591/766] Fetching artist data for SOMI...  done!
[592/766] Fetching artist data for Ali Gatie...  done!
[593/766] Fetching artist data for Reneé Dominique...  done!
[594/766] Fetching artist data for Donnalyn Bartolome...  done!
[595/766] Fetching artist data for Ashley O...  done!
[596/766] Fetching artist data for J Boog...  done!
[597/766] Fetching artist data for Blanco Brown...  done!
[598/766] Fetching artist data for VVS Collective...  done!
[599/766] Fetching artist data for Arvey...  done!
[600/766] Fetching artist data for King Badger...  done!
[601/766] Fetching artist data for Stephanie Poetri...  done!
[602/766] Fetching artist data for Y2K...  done!
[603/766] Fetching artist data for Psychedelic Boyz...  done!
[604/766] Fetching artist data for Kina...  done!
[605/766] Fetching artist data for Lil Tecca...  done!
[606/766] Fetching artist data for Skusta Clee...  do

[736/766] Fetching artist data for Vedo...  done!
[737/766] Fetching artist data for Logan Harris...  done!
[738/766] Fetching artist data for Julian Sean...  done!
[739/766] Fetching artist data for Gagong Rapper...  done!
[740/766] Fetching artist data for Internet Money...  done!
[741/766] Fetching artist data for Jom...  done!
[742/766] Fetching artist data for Claire Rosinkranz...  done!
[743/766] Fetching artist data for Ritt Momney...  done!
[744/766] Fetching artist data for Rob Deniel...  done!
[745/766] Fetching artist data for Ebe Dancel...  done!
[746/766] Fetching artist data for David Archuleta...  done!
[747/766] Fetching artist data for Clairo...  done!
[748/766] Fetching artist data for NCT U...  done!
[749/766] Fetching artist data for CARELESS...  done!
[750/766] Fetching artist data for AJ Rafael...  done!
[751/766] Fetching artist data for GFRIEND...  done!
[752/766] Fetching artist data for Maluma...  done!
[753/766] Fetching artist data for aespa...  done!
[754/7

In [24]:
artist_data_df = pd.concat(df_list)
artist_data_df 

,artist_id,artist_name,total_followers,genres,popularity
0,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,28322108,"[dance pop, pop]",88
0,4IWBUUAFIplrNtaOHcJPRM,James Arthur,7279217,"[pop, talent show, uk pop]",86
0,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,17241689,"[dance pop, edm, electropop, pop, pop dance, t...",85
0,04gDigrS5kc9YWfZHwBETP,Maroon 5,28845735,"[pop, pop rock]",88
0,5p7f24Rk5HkUZsaS3BLG5F,Hailee Steinfeld,6407022,"[dance pop, pop, pop dance, post-teen pop]",78
...,...,...,...,...,...
0,6XpaIBNiVzIetEPCWDvAFP,Whitney Houston,6178661,"[dance pop, pop, urban contemporary]",81
0,4QQgXkCYTt3BlENzhyNETg,"Earth, Wind & Fire",2709786,"[disco, funk, jazz funk, motown, quiet storm, ...",76
0,0bBz5bRBkExaej2HxtVfCw,347aidan,121360,[sad rap],73
0,05YVYeV4HxYp5rrWalvuE1,"Grover Washington, Jr.",176775,[jazz funk],73


In [25]:
artist_data_df.to_csv('data/spotify_daily_charts_artists.csv', index=False, encoding='utf-8')

## Resources
- Spotify API reference manual https://developer.spotify.com/documentation/web-api/reference/search/search/